### 505. The Maze II

There is a ball in a `maze` with empty spaces (represented as `0`) and walls (represented as `1`). The ball can go through the empty spaces by rolling up, down, left or right, but it won't stop rolling until hitting a wall. When the ball stops, it could choose the next direction.

Given the `m x n` `maze`, the ball's `start` position and the `destination`

- `start = [startrow, startcol]` 

- `destination = [destinationrow, destinationcol]`

Return the shortest distance for the ball to **stop** at the `destination`. If the ball cannot stop at `destination`, return `-1`.

The distance is the number of empty spaces traveled by the ball from the `start` position (excluded) to the `destination` (included).

You may assume that the borders of the `maze` are all walls.

<ins>Logic<ins>

This can be considered as a *Shortest Path* problem in *Weighted Graph*, i.e., the weight of edges are different since the distance that the ball rolls will be different depending on its starting position.

*Weighted Graph* can be converted to *Unweighted Graph* by considering one single step at a time (**with direction**).

- Initialize *queue* `queue` and *hashmap* `distance`

    4 nodes are intiialized, i.e., combination of starting position and 4 different directions

- Run **BFS**

    Majority of **BFS** follows standard precedure, the following are the differences:

    1. When exploring next position, consider **the next single step only** instead of the position where the ball hits the wall

        Hence, the direction that the ball will move can be splited into the following cases

    - **Case1: The ball hits the wall**

        $\Rightarrow$ the ball's next position (**using current directions**) is not an empty space

        $\Rightarrow$ the ball can move in **directions that are perpendicular to current direction** (since only these 2 directions can possibly explore new path)
        
    
    - **Case2: The ball is still rolling**

        $\Rightarrow$ the next position using current directions is an empty space

        $\Rightarrow$ the ball can only move in current direction

    2. When determining whether the ball stops at `destination`, we need the position aligns and the ball stops

<br>

Time Complexity: $O(m \times n)$

Space Complexity: $O(m \times n)$

<br>

<ins>**Note:**</ins>

In addition to use **BFS** to find the shortest path, we can also use **BFS/DFS** to get compute the distances of all the paths to `destination` and find the shortest among them


In [48]:
from collections import deque

def is_empty(maze, row, col):
    nrow, ncol = len(maze), len(maze[0])

    return (
        0 <= row < nrow and
        0 <= col < ncol and
        maze[row][col] == 0
    )

def get_positions(maze, row, col, direct, movements):
    result = {}
    for direct_next, (drow, dcol) in movements.items():
        row_next, col_next = row + drow, col + dcol
        # add to result if it is an empty cell
        if is_empty(maze, row_next, col_next):
            result[direct_next] = (row_next, col_next)
    
    # if direct in result -> ball is still rolling
    if direct in result:
        return {direct: result[direct]}

    # return all possibilities if hit wall
    return result
    

def shortestDistance(maze, start, destination):
    # constant
    movements = {
        'up': (-1, 0),
        'down': (1, 0),
        'left': (0, -1),
        'right': (0, 1)
    }

    # init
    queue = deque()
    distance = {}
    for direct in movements.keys():
        queue.append((*start, direct))
        distance[(*start, direct)] = 0

    # bfs
    while queue:
        row, col, direct = queue.popleft()
        curr_distance = distance[(row, col, direct)]
        
        # get next positions
        next_positions = get_positions(maze, row, col, direct, movements)

        # when ball stops, check if reach destination
        if direct not in next_positions and [row, col] == destination:
            return curr_distance
        
        # explore next positions
        for direct_next, (row_next, col_next) in next_positions.items():
            # skip if the position with same direction has been visited
            if (row_next, col_next, direct_next) in distance:
                continue
            
            # update queue and visited
            queue.append((row_next, col_next, direct_next))
            distance[(row_next, col_next, direct_next)] = curr_distance + 1
        
    return -1

12